In [246]:
import os
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt

In [247]:
src1 = "dataset/frlg/"
src2 = "dataset/frlg_cont/"

dst = "dataset/frlg_hue/"
os.makedirs(dst, exist_ok=True)

imgs = os.listdir(src1)
if "back" in imgs:
    imgs.remove("back")

#shffle imgs
import random
random.shuffle(imgs)

for i in imgs:
    toTensor = transforms.ToTensor()
    img = Image.open(src1 + i)
    img_rgb = toTensor(img.convert("RGBA"))
    img_hsv = toTensor(img.convert("HSV"))

    con = Image.open(src2 + i)
    con = toTensor(con.convert("L"))

    white = (img_hsv[[1]] < 0.1).float()
    black = (img_hsv[[2]] < 0.1).float()

    img_hsv[[1]] = 0.8
    img_hsv[[2]] = 0.8

    alpha = img_rgb[[3]]

    conv1 = transforms.ToPILImage(mode="HSV")
    img_rgb = toTensor(conv1(img_hsv).convert("RGB"))

    img_rgb += white
    img_rgb = img_rgb.clamp(0,1)

    img_rgb -= black
    img_rgb = img_rgb.clamp(0,1)

    img_rgb -= con
    img_rgb = img_rgb.clamp(0,1)

    img_rgb += (1-alpha)
    img_rgb = img_rgb.clamp(0,1)

    conv2 = transforms.ToPILImage(mode="RGB")
    conv2(img_rgb).save(dst + i)
